In [1]:
from dotenv import load_dotenv
load_dotenv()

True

El modelo con las tools podra decidir si ejecuta otra tool o responde directamente al usuario.  
Para seguir en el bucle, el modelo debe recibir en cada paso el historial de la conversacion.  
Si no es modelo razonador va a ver como responde pero no va a ser tan bueno como un modelo que razona por si mismo.

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5-nano", temperature=0)
response = llm.invoke('Hola soy Nicolas y mi telefono es +57 317 867 2615')
response.text

In [ ]:
response = llm.invoke("Que clima hace en la ciudad de Bogota?")  # aqui responde con informacion inventada ya que no tiene acceso a internet o un api de clima en tiempo real
response.text

In [ ]:
messages = [
    ("user", "Dime los productos que ofreces en la tienda")
]
response = llm.invoke(messages)
response.text # aqui me inventa los productos que ofrezco en la tienda

In [ ]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan.

Y tus productos son:
- Computadora
- Mouse
- Teclado
- Audifonos
- Mousepad
"""
# o vectorizar los productos ponerlos en una rag que sera lo mismo

messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
response = llm.invoke(messages)
response.text

In [ ]:
from langchain_core.tools import tool
import requests

# puede ser acceder a una api real, o a una base de datos, un csv, etc
# un api viene con una estructura definida
# este decorador tiene propiedades para ayudar al modelo a entender mejor la herramienta y cuando usarla

@tool("get_products", description="Get the products that the store offers filter by price")
def get_products_price(price: float = None):
    """Get the products that the store offers"""
    productus = [
        {"title": "Computadora", "price": 1000},
        {"title": "Mouse", "price": 50},
        {"title": "Teclado", "price": 80},
        {"title": "Audifonos", "price": 150},
        {"title": "Mousepad", "price": 20},
    ]
    return "".join(
        [
            f"{product['title']} - {product['price']}\n"
            for product in productus
            if price is None or product["price"] <= price
        ]
    )



@tool("get_products", description="Get the products that the store offers filter by price")
def get_products():
    # Connnect with API
    """Get the products that the store offers"""
    response = requests.get("https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])

In [10]:
get_products_price.invoke({"price": 70})  # llama a la herramienta directamente

'Mouse - 50\nMousepad - 20\n'

In [6]:
get_products.invoke({})

'Classic Blue Baseball Cap - 186Classic Red Baseball Cap - 35Classic Olive Chino Shorts - 84Classic High-Waisted Athletic Shorts - 43Classic White Crew Neck T-Shirt - 39Classic White Tee - Timeless Style and Comfort - 73Classic Black T-Shirt - 35Sleek White & Orange Wireless Gaming Controller - 69Sleek Wireless Headphone & Inked Earbud Set - 44Sleek Comfort-Fit Over-Ear Headphones - 28Efficient 2-Slice Toaster - 48Sleek Wireless Computer Mouse - 10Sleek Modern Laptop with Ambient Lighting - 43Sleek Modern Laptop for Professionals - 97Stylish Red & Silver Over-Ear Headphones - 39Sleek Mirror Finish Phone Case - 27Sleek Smartwatch with Vibrant Display - 16Sleek Modern Leather Sofa - 53Mid-Century Modern Wooden Dining Table - 24Elegant Golden-Base Stone Top Dining Table - 66Modern Elegance Teal Armchair - 25Elegant Solid Wood Dining Table - 67Modern Minimalist Workstation Setup - 49Modern Ergonomic Office Chair - 71Futuristic Holographic Soccer Cleats - 39Rainbow Glitter High Heels - 39Ch

In [16]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    # de acuerdo al nombre de la ciudad, obtener latitud y longitud
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    # print(data)
    # response = ""
    response = f"The weather in {city} is {data['current_weather']['temperature']}C with {data['current_weather']['windspeed']}km/h of wind."
    return response

get_weather.invoke({"city": "bogota"})

'The weather in bogota is 17.4C with 2.6km/h of wind.'

In [ ]:
# Como se incorporan las tools al modelo
# es bueno en el system prompt explicar que tools tiene y para que sirven
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
# añadimos las tools al modelo
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls # aqui responde que debe usar la herramienta get_products para responder a la pregunta

[{"name": 'get_products',  
'args': {},  
'id': 'toolu_034fsdfgnloip',  
'type': 'tool_call'}]

In [ ]:
response.pretty_print()

In [ ]:
messages = [
    ("system", system_prompt),
    ("user", "Hola, que tal?")
]
response = llm_with_tools.invoke(messages)
response.text # aqui responde saludando pero con tool_calls 
# el modelo traduce en si, si los tools responden en ingles y el usuario en español el modelo responde en el mismo idioma del usuario, ya que el interpreta en el idioma que sea.

In [ ]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima en la capital de Colombia?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls # responde no solo con la tool que debe usar, sino que tambien con los parametros que debe enviar a la tool


[{"name": 'get_wather',  
'args': {'city': 'bogota'},  
'id': 'toolu_034fsdfgdfgdfgdf',  
'type': 'tool_call'}]

otra forma de representar el bind tool es:

y eso se enviaria como `response = self.llm.bind_tools(BANKING_TOOLS).invoke(messages)`

In [ ]:
BANKING_TOOLS = [
    # ==================== CUENTAS ====================
    {
        "type": "function",
        "function": {
            "name": "consultar_estado_cuenta",
            "description": "Obtiene información general de la cuenta bancaria del usuario: tipo, saldo, límite diario, fecha de apertura y estado",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "consultar_saldo",
            "description": "Consulta únicamente el saldo actual de la cuenta",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }]